In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
%%capture
!pip install pandas langchain langchain_experimental langchain-groq
!pip install langchain-huggingface
!pip install -qU trl Py7zr auto-gptq optimum
!pip install llama-index-llms-langchain
!pip install faiss-gpu
!pip install ragas
!pip install llama-index-vector-stores-faiss
!pip install llama-index-readers-file
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-instructor

In [5]:
!pip install "unstructured[all-docs]"
!pip install -q unstructured
!apt-get install -y -q poppler-utils
!pip install unstructured-client watermark
!pip install reportlab
!pip install PyPDF2
!pip install pymupdf pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.3 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.0 MB/s eta 0

In [6]:
import sys
import os
from time import time
import logging
import torch
import shutil
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig, pipeline, TextStreamer, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from datasets import load_dataset, Dataset
from pathlib import Path
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.postprocessor import LLMRerank
from IPython.display import Markdown, display
from llama_index.core import Settings
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore
from IPython.display import Markdown, display
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
import pandas as pd
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')
from IPython.display import JSON
import json
from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import dict_to_elements, elements_to_json
import unstructured.partition
from unstructured.partition.pdf import partition_pdf
from langchain.schema import Document
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
import nest_asyncio
import glob
from IPython.display import Image, display

nest_asyncio.apply()

In [ ]:
!huggingface-cli login --token Your HF Key

groq_api = Groq API Key
llm = ChatGroq(temperature=0, model="gemma2-9b-it", api_key=groq_api)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `Shadman114` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Shadman114`


# Table Data Extraction

In [17]:
filename = "/kaggle/input/testing-manual/Transformer Testing Manual.pdf"

In [6]:
elements = partition_pdf(filename=filename,
                         infer_table_structure=True,
                         strategy='hi_res',
           )

element_dict = [el.to_dict() for el in elements]
output = json.dumps(element_dict, indent=2)

unique_types = set()
for item in element_dict:
    unique_types.add(item['type'])
tables = [el for el in elements if el.category == "Table"]

prompt = ChatPromptTemplate.from_messages(
    [("system", "You will throughly explain the numbers/ row components of table \\n\\n{context} with every details. It is important to mention every number. Do not avoid any row. Suppose column head is 'Person', 'Hobby'. And a row is 'A', 'Sleep'. So you will write it as Person A hobby is to sleep." 
     )])

pikepdf C++ to Python logger bridge initialized


yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

Reading PDF for file: /kaggle/input/testing-manual/Transformer Testing Manual.pdf ...
Loading the Table agent ...
Loading the table structure model ...


config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

Loading pretrained weights from Hugging Face hub (timm/resnet18.a1_in1k)


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

[timm/resnet18.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.


In [7]:
table_html = tables[0].metadata.text_as_html
from IPython.core.display import HTML
HTML(table_html)

Transformer winding voltage (kV),Factory test AC voltage (kV),"Acceptance field test AC voltage, 75% (kV)","Maintenance periodic test, 65% (kV)"
1.20,10,7.50,6.50
2.40,15,11.20,9.75
4.80,19,14.25,12.35
8.70,26,19.50,16.90
15.00,34,25.50,22.10
18,40,30.00,26.00
25.00,50,37.50,32.50
34.50,70,52.50,45.50
46.00,95,71.25,61.75
69.00,140,105.00,91.00


In [8]:
# Function to save results to PDF with font size 8
def save_results_to_pdf(results, filename="results.pdf"):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter

    cursor_y = height - 40  # Start below the top margin

    for i, result in enumerate(results):
        # Write table result heading
        c.setFont("Helvetica-Bold", 8)
        c.drawString(40, cursor_y, f"Result for Table {i + 1}:")
        cursor_y -= 15  # Move cursor down slightly

        # Write the result content line by line
        c.setFont("Helvetica", 8)
        lines = result.split('\n')  # Handle multi-line content

        for line in lines:
            if cursor_y < 40:  # Create a new page if space runs out
                c.showPage()
                cursor_y = height - 40
                c.setFont("Helvetica", 8)  # Reset font for the new page

            # Break long lines to fit within page width
            while len(line) > 90:  # Assume ~90 chars per line
                part = line[:90]  # Take the first 90 characters
                c.drawString(40, cursor_y, part)
                cursor_y -= 12
                line = line[90:]  # Continue with the remaining part

            c.drawString(40, cursor_y, line)
            cursor_y -= 12  # Move cursor down for the next line

        cursor_y -= 20  # Add space between table results

    c.save()
    print(f"Results saved to {filename}")

# Loop through each table and apply the LLM chain
results = []  # Collect all results

for i, table in enumerate(tables):
    print(f"\nProcessing Table {i + 1}...\n")

    # Safely access HTML content from metadata
    table_html = table.metadata.text_as_html

    if table_html:
        # Create a LangChain Document object with table content
        context = [Document(page_content=table_html)]
        chain = create_stuff_documents_chain(llm, prompt)

        # Invoke the chain for this table
        result = chain.invoke({"context": context})

        # Collect result
        results.append(result)
        print(f"Result for Table {i + 1}:")
        print(result)
    else:
        print(f"HTML content not found for Table {i + 1}.")
        results.append(f"HTML content not found for Table {i + 1}.")

# Save all results to a PDF
#save_results_to_pdf(results, "table_results.pdf")
os.makedirs("/kaggle/working/Documents", exist_ok=True)
save_results_to_pdf(results, "/kaggle/working/Documents/table_results.pdf")


Processing Table 1...

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Result for Table 1:
Let's break down the table row by row:

**Row 1:**

* **Transformer winding voltage (kV):** 1.20 
* **Factory test AC voltage (kV):** 10
* **Acceptance field test AC voltage, 75% (kV):** 7.50
* **Maintenance periodic test, 65% (kV):** 6.50

This row tells us that a transformer with a winding voltage of 1.20 kV undergoes a factory test with an AC voltage of 10 kV.  For acceptance field testing, the voltage is 75% of the factory test voltage (7.50 kV), and for periodic maintenance, it's 65% (6.50 kV).

**Row 2:**

* **Transformer winding voltage (kV):** 2.40
* **Factory test AC voltage (kV):** 15
* **Acceptance field test AC voltage, 75% (kV):** 11.20
* **Maintenance periodic test, 65% (kV):** 9.75

Here, a transformer with a winding voltage of 2.40 kV is tested at 15 kV in the factory.  Acceptance field testing uses 75% of the factory test voltage (11.20 kV), 

# Image Extraction

In [7]:
#filename = "/kaggle/input/testing-manual/Power Cable Testing Manual.pdf"

In [8]:
# # Get elements
# path = "/kaggle/working/Images_from_PDF"
# #filename = "/kaggle/input/testing-manual/Power Cable Testing Manual.pdf"
# raw_pdf_elements = partition_pdf(filename=filename,
#                                  # Unstructured first finds embedded image blocks
#                                  # Only applicable if `strategy=hi_res`
#                                  extract_images_in_pdf=True,
#                                  strategy = "hi_res",
#                                  infer_table_structure=True,
#                                  # Only applicable if `strategy=hi_res`
#                                  extract_image_block_output_dir = path,
#                                  )

# element_dict = [el.to_dict() for el in raw_pdf_elements]

# unique_types = set()

# for item in element_dict:
#     unique_types.add(item['type'])

# print(unique_types)
# images = [el for el in raw_pdf_elements if el.category == "Image"]
# len(images)

pikepdf C++ to Python logger bridge initialized


yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

Reading PDF for file: /kaggle/input/testing-manual/Power Cable Testing Manual.pdf ...
Loading the Table agent ...
Loading the table structure model ...


config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

Loading pretrained weights from Hugging Face hub (timm/resnet18.a1_in1k)


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

[timm/resnet18.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
{'ListItem', 'Image', 'Table', 'Title', 'FigureCaption', 'NarrativeText', 'Formula'}


1

In [16]:
# #Define the path to the folder containing the images
# image_path = "/kaggle/working/Images_from_PDF/figure-18-31.jpg"  # Update the file type as needed

# # Use glob to search for JPG files in the specified folder
# image_files = glob.glob(image_path)

# # Iterate through the list of image files and display each image inline
# for image_file in image_files:
#     display(Image(filename=image_file))

In [ ]:
# import os
# import base64
# from groq import Groq

# # Initialize Groq client
# client = Groq(api_key=GROQ API Key)

# # Function to encode the image
# def encode_image(image_path):
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode('utf-8')

# # Specific image path
# image_path = '/kaggle/working/Images_from_PDF/figure-18-31.jpg'

# if os.path.isfile(image_path):  # Check if the file exists
#     print(f"\nProcessing Image: {image_path}...\n")

#     # Encode the image
#     base64_image = encode_image(image_path)

#     # Create chat completion request
#     chat_completion = client.chat.completions.create(
#         messages=[
#             {
#                 "role": "user",
#                 "content": [
#                     {"type": "text", "text": "This image will be used in Retrieval Augmented Generation. So explain it as details as you can"},
#                     {
#                         "type": "image_url",
#                         "image_url": {
#                             "url": f"data:image/jpeg;base64,{base64_image}",
#                         },
#                     },
#                 ],
#             }
#         ],
#         model="llama-3.2-90b-vision-preview",
#     )

#     result = chat_completion.choices[0].message.content
#     print(f"Result for Image {image_path}:")
#     print(result)


In [18]:
# from groq import Groq
# import os
# import base64

In [ ]:
# # Initialize Groq client
# client = Groq(api_key=GROQ API Key)

# # Function to encode the image
# def encode_image(image_path):
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode('utf-8')

# # Function to save results to PDF with font size 8
# def save_results_to_pdf(results, filename="results.pdf"):
#     c = canvas.Canvas(filename, pagesize=letter)
#     width, height = letter

#     cursor_y = height - 40  # Start below the top margin

#     for i, result in enumerate(results):
#         # Write table result heading
#         c.setFont("Helvetica-Bold", 8)
#         c.drawString(40, cursor_y, f"Result for Image {i + 1}:")
#         cursor_y -= 15  # Move cursor down slightly

#         # Write the result content line by line
#         c.setFont("Helvetica", 8)
#         lines = result.split('\n')  # Handle multi-line content

#         for line in lines:
#             if cursor_y < 40:  # Create a new page if space runs out
#                 c.showPage()
#                 cursor_y = height - 40
#                 c.setFont("Helvetica", 8)  # Reset font for the new page

#             # Break long lines to fit within page width
#             while len(line) > 90:  # Assume ~90 chars per line
#                 part = line[:90]  # Take the first 90 characters
#                 c.drawString(40, cursor_y, part)
#                 cursor_y -= 12
#                 line = line[90:]  # Continue with the remaining part

#             c.drawString(40, cursor_y, line)
#             cursor_y -= 12  # Move cursor down for the next line

#         cursor_y -= 20  # Add space between image results

#     c.save()
#     print(f"Results saved to {filename}")

# # Directory containing images
# images_folder = '/kaggle/working/Images_from_PDF'
# results = []  # Collect all results

# # Loop through each image in the folder
# for image_name in os.listdir(images_folder):
#     image_path = os.path.join(images_folder, image_name)

#     if os.path.isfile(image_path):  # Check if it's a file
#         print(f"\nProcessing Image: {image_name}...\n")
        
#         # Encode the image
#         base64_image = encode_image(image_path)

#         # Create chat completion request
#         chat_completion = client.chat.completions.create(
#             messages=[
#                 {
#                     "role": "user",
#                     "content": [
#                         {"type": "text", "text": "This image will be used in Retrieval Augmented Generation. So explain it as details as you can"},
#                         {
#                             "type": "image_url",
#                             "image_url": {
#                                 "url": f"data:image/jpeg;base64,{base64_image}",
#                             },
#                         },
#                     ],
#                 }
#             ],
#             model="llama-3.2-90b-vision-preview",
#         )
        
#         result = chat_completion.choices[0].message.content
#         results.append(result)
#         print(f"Result for Image {image_name}:")
#         print(result)

# # Save all results to a PDF
# save_results_to_pdf(results, "image_results.pdf")


# LLM 

In [9]:
import os
# Step 3: Define source and destination paths
source_path = "/kaggle/input/testing-manual/Transformer Testing Manual.pdf"
destination_dir = "/kaggle/working/Documents"
destination_path = os.path.join(destination_dir, "Transformer Testing Manual.pdf")

# Step 4: Check if destination directory exists, if not, create it
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Step 5: Copy the PDF file to the destination folder
shutil.copy(source_path, destination_path)

# Step 6: Verify if the file has been copied successfully
if os.path.exists(destination_path):
    print("File copied successfully!")
else:
    print("File copy failed.")


# Define the source and destination paths
source_dir = '/kaggle/input/gemma-ft/Gemma_FT'
destination_dir = '/kaggle/working/FineTuned'

# Copy all contents (files and subdirectories) from source to destination
shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)

File copied successfully!


'/kaggle/working/FineTuned'

In [18]:
model_ft = "marcsun13/gemma-2-9b-it-GPTQ"

tokenizer_ft = AutoTokenizer.from_pretrained(model_ft)

output_dir_new = "/kaggle/working/FineTuned/finetuned_raft_model"

In [19]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)

# disable_exllama=True

model_ft = AutoModelForCausalLM.from_pretrained(output_dir_new,quantization_config=quantization_config_loading, device_map="auto")

pipe_ft = pipeline(
    "text-generation",
    model=model_ft,
    tokenizer=tokenizer_ft,
    max_new_tokens=500,
    do_sample=True,
    temperature=0.01,
    top_p=1,
    top_k=5,
    repetition_penalty=1.1
)

Settings.llm = HuggingFacePipeline(pipeline=pipe_ft)
Settings.chunk_size = 512

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at marcsun13/gemma-2-9b-it-GPTQ were not used when initializing Gemma2ForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11.mlp.gate_p

In [20]:
doc = SimpleDirectoryReader("/kaggle/working/Documents").load_data()

# Retrieve 

In [21]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# dimensions of text-ada-embedding-002
d = 384
faiss_index = faiss.IndexFlatL2(d)

In [22]:
# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model

Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [23]:
vector_store = FaissVectorStore(faiss_index=faiss_index)  # Pass the embed_model

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    doc, storage_context=storage_context
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
pd.set_option("display.max_colwidth", None)

def get_retrieved_nodes(
    query_str, vector_top_k=10, reranker_top_n=3, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = LLMRerank(
            choice_batch_size=5,
            top_n=reranker_top_n,
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "<br>")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

In [25]:
new_nodes = get_retrieved_nodes(
    "For 1.20kV Transformer winding voltage what is Factory test AC voltage",
    vector_top_k=5,
    reranker_top_n=3,
    with_reranker=True,
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


In [26]:
visualize_retrieved_nodes(new_nodes)

,Score,Text
0,10.0,"Result for Table 1:Let's break down the table row by row:**Row 1:*** **Transformer winding voltage (kV):** 1.20 * **Factory test AC voltage (kV):** 10* **Acceptance field test AC voltage, 75% (kV):** 7.50* **Maintenance periodic test, 65% (kV):** 6.50This row tells us that a transformer with a winding voltage of 1.20 kV undergoes a factory test with an AC voltage of 10 kV. For acceptance field testing, the voltage is 75% of the factory test voltage (7.50 kV), and for periodic maintenance, it's 65% (6.50 kV).**Row 2:*** **Transformer winding voltage (kV):** 2.40* **Factory test AC voltage (kV):** 15* **Acceptance field test AC voltage, 75% (kV):** 11.20* **Maintenance periodic test, 65% (kV):** 9.75Here, a transformer with a winding voltage of 2.40 kV is tested at 15 kV in the factory. Acceptance field testing uses 75% of the factory test voltage (11.20 kV), and maintenance testing uses 65% (9.75 kV).**Row 3:*** **Transformer winding voltage (kV):** 4.80* **Factory test AC voltage (kV):** 19* **Acceptance field test AC voltage, 75% (kV):** 14.25* **Maintenance periodic test, 65% (kV):** 12.35The pattern continues: a 4.80 kV transformer is factory tested at 19 kV, with acceptance field testing at 75% (14.25 kV) and maintenance testing at 65% (12.35 kV)."
1,4.0,"POWER TRANSFORMER TESTING Transformers may be tested using AC or DC voltage. AC voltage is preferable to DC voltage for transformer testing because AC voltage simulates the internal stress that the transformers face during operating conditions. The following tests are routinely conducted in the field on the transformer: - Excitation current test - Insulating fluid dielectric tests - Insulation PF test - Insulation Resistance test - TTR test - Polarity test - AC or DC hi-pot test - Induced potential test - Frequency response analyzer - Dissolved gas analysis tests - DC winding resistance - Transformer core ground test - Polarization recovery voltage test AC Hi-Pot Test The AC hi -pot test is used to assess transformer windings condition. Hi-pot test is suggested for all voltages, particularly those above 34.5 kV. For transformer routine"


In [27]:
query_engine = index.as_query_engine(
    similarity_top_k=4,
    node_postprocessors=[
        LLMRerank(
            choice_batch_size=5,
            top_n=2,
        )
    ],
    response_mode="compact",
)

In [14]:
response = query_engine.query(
    "What should be done to complete the TTR test?",
)
print(response)
answer = response.response        # The actual answer

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


Context information is below.
---------------------
page_label: 4
file_path: /kaggle/working/Documents/Transformer Testing Manual.pdf

The TTR test gives the following: 
- It checks the turns ratio and polarity o f single - and three -phase power 
transformers, one phase at a time. 
- It verifies nameplate ratio, polarity, and vectors. 
- It checks polarity and the ratio (but not voltage rating) of transformers without 
markings. Tests consider all transformer no-load tap positions. Tests consider 
all load taps on load, tap change r (LTC) transformers if connected for voltage 
ratio control. On LTC transformers connected for phase angle control, r atio 
and polarity are completed only in neutral positions. If checked on load taps, 
measurements may be taken for reference for future compariso n, but will 
deviate from nameplate ratings. LTC taps may be checked by using low three-
phase voltage and reading volts and the phase angle for each. 
- Find issues in power transformer win dings

In [ ]:
#context, question, answer= extract_context_question_answer(answer)

In [ ]:
#print(context)
#print(question)
#print(answer)

# Evaluation

In [ ]:
#!pip install datasets==2.14.5 cohere==4.27 chromadb langchain sentence-transformers ragas

In [28]:
def extract_context_question_answer(data):
    # Step 1: Extract context
    context_end_keyword = "Query:"
    context = data.split(context_end_keyword)[0].strip()

    # Step 2: Extract question and answer
    question_answer = data.split(context_end_keyword)[1].strip()

    # Separate question and answer
    answer_start_keyword = "Answer:"
    question = question_answer.split(answer_start_keyword)[0].strip()
    answer = question_answer.split(answer_start_keyword)[1].strip()

    return context, question, answer

In [29]:
questions = ["Why AC hi-pot test is used",
             "What should be done to complete the TTR test?",
             "A transformer needs to verify the continuity and phase identification of windings. What tests should be done?",
             "Why AC voltage is preferred for transformer testing?",
             "What are the common internal mechanical issues found in transformers with FRA are"]

answers = []
contexts = []

question_counter = 0

# Inference
for question in questions:
    response = query_engine.query(question)

    answer_RAG = response.response        # The actual answer

    # Extract context, question, and answer
    context, question, answer = extract_context_question_answer(answer_RAG)

    answers.append(answer)
    #print(answers)
    contexts.append([context])
    #print(contexts)
    # Increment the counter after processing each question
    question_counter += 1

    # Print or log the current count
    print(f"Processed question number: {question_counter}")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processed question number: 1


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processed question number: 2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processed question number: 4


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processed question number: 5


In [30]:
ground_truths = [ "The AC hi-pot test is used to assess transformer windings condition.",
                 "The following steps are used for completing the TTR test: \n- Transformer is isolated and tagged and leads disconnected \n- Check transformer nameplate \n- Check the polarities and vectors (phasors) \n- Determine ratios for each no-load and load tap position",
                 "Test 1: Connect the 120 V, 60 Hz power through the lamp to the transformer primary, terminals. Leave the transformer secondary winding open. The lamp will burn dimly. \n Test 2: Keep connections as presented in test 1, but now short the secondary winding. The lamp should burn with great brilliance. If the lamp still burns with somewhat less than full brilliance, check for issues in the transformer winding. \n Test 3: This test is similar to tests 1 and 2, but as applied to a three phase transformer for phase identification and phase continuity check. Complete tests 1 and 2 for each winding of a three-phase transformer individually with the remaining windings kept open.",
                 "AC voltage simulates the internal stress that the transformers face during operating conditions.",
                 "- Core displacement,\n- Partial winding collapse,\n- Faulty core grounds,\n- Shorted turns and open windings,\n- Broken or loosened clamping structures,\n- Winding deformation and displacement"]

data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "reference": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)
print(dataset)


# To dict
data_1 = {
    "question": questions[0:1],
    "answer": answers[0:1],
    "contexts": contexts[0:1],
    "reference": ground_truths[0:1]
}

# Convert dict to dataset
dataset_1 = Dataset.from_dict(data_1)
print(dataset_1)

data_2 = {
    "question": questions[1:2],
    "answer": answers[1:2],
    "contexts": contexts[1:2],
    "reference": ground_truths[1:2]
}

# Convert dict to dataset
dataset_2 = Dataset.from_dict(data_2)
print(dataset_2)

data_3 = {
    "question": questions[2:3],
    "answer": answers[2:3],
    "contexts": contexts[2:3],
    "reference": ground_truths[2:3]
}

# Convert dict to dataset
dataset_3 = Dataset.from_dict(data_3)
print(dataset_3)

data_4 = {
    "question": questions[3:4],
    "answer": answers[3:4],
    "contexts": contexts[3:4],
    "reference": ground_truths[3:4]
}

# Convert dict to dataset
dataset_4 = Dataset.from_dict(data_4)
print(dataset_4)

data_5 = {
    "question": questions[4:5],
    "answer": answers[4:5],
    "contexts": contexts[4:5],
    "reference": ground_truths[4:5]
}

# Convert dict to dataset
dataset_5 = Dataset.from_dict(data_5)
print(dataset_5)

Dataset({
    features: ['question', 'answer', 'contexts', 'reference'],
    num_rows: 5
})
Dataset({
    features: ['question', 'answer', 'contexts', 'reference'],
    num_rows: 1
})
Dataset({
    features: ['question', 'answer', 'contexts', 'reference'],
    num_rows: 1
})
Dataset({
    features: ['question', 'answer', 'contexts', 'reference'],
    num_rows: 1
})
Dataset({
    features: ['question', 'answer', 'contexts', 'reference'],
    num_rows: 1
})
Dataset({
    features: ['question', 'answer', 'contexts', 'reference'],
    num_rows: 1
})


# GPT Evaluation

In [26]:
# Convert to DataFrame and save as CSV for GPT Evaluation
df = pd.DataFrame(data)
df.to_csv("transformer_test_dataset.csv", index=False)
print("Dataset saved as transformer_test_dataset.csv")

Dataset saved as transformer_test_dataset.csv


# GPT EVALUATION SCORE

Here’s the individual evaluation for each answer:

    Why AC hi-pot test is used
        Faithfulness: 0.73
        Context Recall: 0.56

    What should be done to complete the TTR test?
        Faithfulness: 1.00
        Context Recall: 0.58

    A transformer needs to verify the continuity and phase identification…
        Faithfulness: 0.70
        Context Recall: 0.72

    Why AC voltage is preferred for transformer testing?
        Faithfulness: 0.78
        Context Recall: 0.29

    What are the common internal mechanical issues detected during testing?
        Faithfulness: 1.00
        Context Recall: 0.27

    Avarage : 
        Faithfulness: 0.842
        Context Recall: 0.484

In [27]:
# #print(ground_truths)
# # Assuming 'answers' is your list
# for answer in answers:
#     print(answer)

# #print(contexts)

# RAGas Evaluation

In [18]:
import pandas as pd
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)


In [19]:
embedding = HuggingFaceEmbeddings()

Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Use pytorch device_name: cuda
Use pytorch device_name: cuda


In [20]:
llm_RAG = ChatGroq(temperature=0, model="llama-3.3-70b-versatile", api_key=groq_api)

In [21]:
df_1 = pd.DataFrame()

result_1 = evaluate(
    dataset = dataset_1,
    llm=llm_RAG,
    embeddings=embedding,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ],
)

df_1 = result_1.to_pandas()

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completion

In [22]:
df_1

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,Why AC hi-pot test is used,"[Context information is below.\n---------------------\npage_label: 2\nfile_path: /kaggle/working/Documents/Transformer Testing Manual.pdf\n\nPOWER TRANSFORMER TESTING \nTransformers may be tested using AC or DC voltage. AC voltage is preferable to DC \nvoltage for transformer testing because AC voltage simulates the internal stress that \nthe transformers face during operating conditions. The following tests are routinely \nconducted in the field on the transformer: \n- Excitation current test \n- Insulating fluid dielectric tests \n- Insulation PF test \n- Insulation Resistance test \n- TTR test \n- Polarity test \n- AC or DC hi-pot test \n- Induced potential test \n- Frequency response analyzer \n- Dissolved gas analysis tests \n- DC winding resistance \n- Transformer core ground test \n- Polarization recovery voltage test \nAC Hi-Pot Test \nThe AC hi -pot test is used to assess transformer windings condition. Hi-pot test is \nsuggested for all voltages, particularly those above 34.5 kV. For transformer routine\n\npage_label: 3\nfile_path: /kaggle/working/Documents/Transformer Testing Manual.pdf\n\nmaintenance testing, the test voltage should not surpass 65% of factory test voltage. \nNevertheless, the hi -pot test for routine maintenance is typically not applied to \ntransformers because of the possibility of damage to the winding insulation. Hi-pot \ntest is typically used for acceptance testing or after transformer repair testing. The \nAC HV test value should not surpass 75% of the factory test value. When AC hi-pot \ntests are used for routine maintenance, the power transformer can be examined at \nrated voltage for 3 min instead of testing at 65% of factory test voltage. The AC hi -\npot test values for voltages up to 69 kV are presented in Table 1. \nTable 1. AC dielectric test for acceptance and routine maintenance for liquid-filled \npower transformers \nTransformer \nwinding voltage \n(kV) \nFactory test AC \nvoltage (kV) \nAcceptance field test \nAC voltage, \n75% (kV) \nMaintenance \nperiodic test, \n65% (kV) \n1.20 10 7.50 6.50 \n2.40 15 11.20 9.75 \n4.80 19 14.25 12.35 \n8.70 26 19.50 16.90 \n15.00 34 25.50 22.10 \n18 40 30.00 26.00 \n25.00 50 37.50 32.50 \n34.50 70 52.50 45.50 \n46.00 95 71.25 61.75 \n69.00 140 105.00 91.00 \n \nTTR Test \nDuring TTR test voltage is applied to one transformer winding. Also, voltage on \nanother winding on the same core is detected. In the case of a low voltage hand -\ncrank powered TTR, 8 V AC is applied to the tested, low-voltage transformer winding \nand a reference transformer in the TTR set. The HV transformer windings and the \nTTR reference transformer are connected through null detecting equipment. After \npolarity has been made at 8 V, when the null reading is zero, the dial readings show \nthe ratio of the tested transformer.\n---------------------\nGiven the context information and not prior knowledge, answer the query.]",AC hi-pot test is used to assess transformer windings condition. It simulates the internal stress that transformers face during operating conditions.,The AC hi-pot test is used to assess transformer windings condition.,0.5,0.748477,1.0,1.0


In [23]:
df_2 = pd.DataFrame()

result_2 = evaluate(
    dataset = dataset_2,
    llm=llm_RAG,
    embeddings=embedding,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ],
)

df_2 = result_2.to_pandas()
df_2

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 6.000000 seconds
Retrying request to /openai/v1/chat/completions in 6.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 7.000000 seconds
Retrying request to /openai/v1/chat/completions in 7.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,What should be done to complete the TTR test?,"[Context information is below.\n---------------------\npage_label: 4\nfile_path: /kaggle/working/Documents/Transformer Testing Manual.pdf\n\nThe TTR test gives the following: \n- It checks the turns ratio and polarity o f single - and three -phase power \ntransformers, one phase at a time. \n- It verifies nameplate ratio, polarity, and vectors. \n- It checks polarity and the ratio (but not voltage rating) of transformers without \nmarkings. Tests consider all transformer no-load tap positions. Tests consider \nall load taps on load, tap change r (LTC) transformers if connected for voltage \nratio control. On LTC transformers connected for phase angle control, r atio \nand polarity are completed only in neutral positions. If checked on load taps, \nmeasurements may be taken for reference for future compariso n, but will \ndeviate from nameplate ratings. LTC taps may be checked by using low three-\nphase voltage and reading volts and the phase angle for each. \n- Find issues in power transformer win dings, such as open -circuit and short-\ncircuits of turn-to-turn sensitivity. The standard deviation as described in IEEE \nC57.12.00-2006, suggests that results should be within 0.5% of namepla te \nmarkings, with rated voltage applied to one winding. The TTR wi th accuracy \nof 0.1% is accepted as sufficiently accurate. \nThe following steps are used for completing the TTR test: \n- Transformer is isolated and tagged and leads disconnected \n- Check transformer nameplate \n- Check the polarities and vectors (phasors) \n- Determine ratios for each no-load and load tap position \nThe test connections are presented in F igure 1 a through c . In the case of an \nelectronic TTR, a unity ratio check is also done, but null and zero checks are not \nmandatory.\n\npage_label: 5\nfile_path: /kaggle/working/Documents/Transformer Testing Manual.pdf\n\nFigure 1. TTR test set connections; (b) test connection for null verification for TTR \nand (c) test connections for zero verification for TTR \nAlternative Test for TTR \nIn case a TTR test set cannot be used , fast and rough test can be completed to \nverify the continuity and phase identifi cation of transformer windings. The test \nconsists of the following. The eq uipment required for this test is a 100 W lamp with \nsocket and an extension cord for connection to a 120 V 60 Hz power supply, with \nwhich three test procedures are completed. \nTest 1: Connect the 120 V, 60 Hz power through the lamp to the transformer \nprimary, terminals as presented in Figure 2 (a). Leave the transformer secondary \nTransformer \nN \nV \nA \nTTR test set \nH2 \n X1 \nX2 \nH2 \n(c) \nTransformer \nN \nV \nA \nTTR test set \nH1 \nX1 \nX1 \nH2 \n(b\n) \nTransformer \nN \nV \nA \nTTR test set \nH\n1 \nX1 \nX2 \nH2 \n(a)\n---------------------\nGiven the context information and not prior knowledge, answer the query.]",The following steps are used for completing the TTR test: \n- Transformer is isolated and tagged and leads disconnected \n- Check transformer nameplate \n- Check the polarities and vectors (phasors) \n- Determine ratios for each no-load and load tap position,The following steps are used for completing the TTR test: \n- Transformer is isolated and tagged and leads disconnected \n- Check transformer nameplate \n- Check the polarities and vectors (phasors) \n- Determine ratios for each no-load and load tap position,1.0,0.933553,1.0,1.0


In [26]:
df_3 = pd.DataFrame()

result_3 = evaluate(
    dataset = dataset_3,
    llm=llm_RAG,
    embeddings=embedding,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ],
)

df_3 = result_3.to_pandas()
df_3

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.groq.com/openai/v1/

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,A transformer needs to verify the continuity and phase identification of windings. What tests should be done?,"[Context information is below.\n---------------------\npage_label: 6\nfile_path: /kaggle/working/Documents/Transformer Testing Manual.pdf\n\nwinding open. The lamp will burn dimly. \nTest 2: Keep connections as presented in test 1, but now short the secondary \nwinding. The lamp should burn with great brilliance. If the lamp still burns with \nsomewhat less than full brilliance, check for issues in the transformer winding. \nConnections for this test are presented in Figure 2 (b). \nTest 3: This test is similar to tests 1 and 2, but as applied to a three phase \ntransformer for phase identification and phase continuity check. Complete tests 1 \nand 2 for each winding of a three -phase transformer individually with the remaining \nwindings kept open. The test connection arrangement is shown in Figure 2 (c). \n \nFigure 2. Transformer winding continuity verification \n \n(c) \n100W \nH1 \nX1 \nX0 \n120V \n60Hz \nX2 \nX3 \nH3 \nH2 \n100W \nH1 \nH2 \nX2 \nX1 \n120V \n60Hz \n(b) \n100W \nH1 \nH2 \nX2 \nX1 \n120V \n60Hz \n(a)\n\npage_label: 5\nfile_path: /kaggle/working/Documents/Transformer Testing Manual.pdf\n\nFigure 1. TTR test set connections; (b) test connection for null verification for TTR \nand (c) test connections for zero verification for TTR \nAlternative Test for TTR \nIn case a TTR test set cannot be used , fast and rough test can be completed to \nverify the continuity and phase identifi cation of transformer windings. The test \nconsists of the following. The eq uipment required for this test is a 100 W lamp with \nsocket and an extension cord for connection to a 120 V 60 Hz power supply, with \nwhich three test procedures are completed. \nTest 1: Connect the 120 V, 60 Hz power through the lamp to the transformer \nprimary, terminals as presented in Figure 2 (a). Leave the transformer secondary \nTransformer \nN \nV \nA \nTTR test set \nH2 \n X1 \nX2 \nH2 \n(c) \nTransformer \nN \nV \nA \nTTR test set \nH1 \nX1 \nX1 \nH2 \n(b\n) \nTransformer \nN \nV \nA \nTTR test set \nH\n1 \nX1 \nX2 \nH2 \n(a)\n---------------------\nGiven the context information and not prior knowledge, answer the query.]","The manual describes three tests using a 100W lamp and a 120V 60Hz power supply to verify transformer winding continuity and phase identification. \n\n* **Test 1:** Connect the power supply to the primary terminals of the transformer while leaving the secondary open. Observe if the lamp lights up dimly.\n* **Test 2:** Short the secondary winding while maintaining the connections from Test 1. The lamp should illuminate brightly.\n* **Test 3:** For three-phase transformers, repeat Tests 1 and 2 for each individual winding while keeping the other windings open.","Test 1: Connect the 120 V, 60 Hz power through the lamp to the transformer primary, terminals. Leave the transformer secondary winding open. The lamp will burn dimly. \n Test 2: Keep connections as presented in test 1, but now short the secondary winding. The lamp should burn with great brilliance. If the lamp still burns with somewhat less than full brilliance, check for issues in the transformer winding. \n Test 3: This test is similar to tests 1 and 2, but as applied to a three phase transformer for phase identification and phase continuity check. Complete tests 1 and 2 for each winding of a three-phase transformer individually with the remaining windings kept open.",1.0,0.963467,1.0,1.0


In [27]:
df_4 = pd.DataFrame()

result_4 = evaluate(
    dataset = dataset_4,
    llm=llm_RAG,
    embeddings=embedding,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ],
)

df_4 = result_4.to_pandas()
df_4

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 6.000000 seconds
Retrying request to /openai/v1/chat/completions in 6.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 7.000000 seconds
Retrying request to /openai/v1/chat/completions in 7.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,Why AC voltage is preferred for transformer testing?,"[Context information is below.\n---------------------\npage_label: 2\nfile_path: /kaggle/working/Documents/Transformer Testing Manual.pdf\n\nPOWER TRANSFORMER TESTING \nTransformers may be tested using AC or DC voltage. AC voltage is preferable to DC \nvoltage for transformer testing because AC voltage simulates the internal stress that \nthe transformers face during operating conditions. The following tests are routinely \nconducted in the field on the transformer: \n- Excitation current test \n- Insulating fluid dielectric tests \n- Insulation PF test \n- Insulation Resistance test \n- TTR test \n- Polarity test \n- AC or DC hi-pot test \n- Induced potential test \n- Frequency response analyzer \n- Dissolved gas analysis tests \n- DC winding resistance \n- Transformer core ground test \n- Polarization recovery voltage test \nAC Hi-Pot Test \nThe AC hi -pot test is used to assess transformer windings condition. Hi-pot test is \nsuggested for all voltages, particularly those above 34.5 kV. For transformer routine\n\npage_label: 1\nfile_path: /kaggle/working/Documents/table_results.pdf\n\nResult for Table 1:\nLet's break down the table row by row:\n**Row 1:**\n* **Transformer winding voltage (kV):** 1.20 \n* **Factory test AC voltage (kV):** 10\n* **Acceptance field test AC voltage, 75% (kV):** 7.50\n* **Maintenance periodic test, 65% (kV):** 6.50\nThis row tells us that a transformer with a winding voltage of 1.20 kV undergoes a factory\n test with an AC voltage of 10 kV. For acceptance field testing, the voltage is 75% of th\ne factory test voltage (7.50 kV), and for periodic maintenance, it's 65% (6.50 kV).\n**Row 2:**\n* **Transformer winding voltage (kV):** 2.40\n* **Factory test AC voltage (kV):** 15\n* **Acceptance field test AC voltage, 75% (kV):** 11.20\n* **Maintenance periodic test, 65% (kV):** 9.75\nHere, a transformer with a winding voltage of 2.40 kV is tested at 15 kV in the factory. \nAcceptance field testing uses 75% of the factory test voltage (11.20 kV), and maintenance \ntesting uses 65% (9.75 kV).\n**Row 3:**\n* **Transformer winding voltage (kV):** 4.80\n* **Factory test AC voltage (kV):** 19\n* **Acceptance field test AC voltage, 75% (kV):** 14.25\n* **Maintenance periodic test, 65% (kV):** 12.35\nThe pattern continues: a 4.80 kV transformer is factory tested at 19 kV, with acceptance f\nield testing at 75% (14.25 kV) and maintenance testing at 65% (12.35 kV).\n---------------------\nGiven the context information and not prior knowledge, answer the query.]",AC voltage is preferable to DC voltage for transformer testing because AC voltage simulates the internal stress that the transformers face during operating conditions.,AC voltage simulates the internal stress that the transformers face during operating conditions.,1.0,0.964333,1.0,1.0


In [28]:
df_5 = pd.DataFrame()

result_5 = evaluate(
    dataset = dataset_5,
    llm=llm_RAG,
    embeddings=embedding,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ],
)

df_5 = result_5.to_pandas()
df_5

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 44.000000 seconds
Retrying request to /openai/v1/chat/completions in 44.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 29.000000 seconds
Retrying request to /openai/v1/chat/completions in 29.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,What are the common internal mechanical issues found in transformers with FRA are,"[Context information is below.\n---------------------\npage_label: 11\nfile_path: /kaggle/working/Documents/Transformer Testing Manual.pdf\n\nThe sweep frequency response analyzer introduces a powerful tool to the quality \ncontrol and maintenance equipment, allowing a look inside the transformer to \ndiscover even subtle changes in the core and winding mechanical structure—without \nemptying the transformer tank . This i s the mo st efficient diagnostic tool for \ndiscovering mechanical issues in pow er transformers. The instrument emits an \nexcitation signal into t he transformer and detects the returning signals. Comparing \nthis response to b aseline and other results (such as from similar units) allows \ndetection of deviations. Common internal mechanical issues found in transformers \nwith FRA are: \n- Core displacement \n- Partial winding collapse \n- Faulty core grounds \n- Shorted turns and open windings \n- Broken or loosened clamping structures \n- Winding deformation and displacement \nDescribed test techniques are typically used on big HV power transformers because \nthey are sensitive tests to discover winding d istortion and deformation in power \ntransformers. Considerable amount of deformation can happen in the windings as a \nresult of high through fault currents which can remain undiscovered before an actual \nfailure happens. The voltage stress changes in the winding insulation structure after \nthe onset of initial winding deformation. Subsequently, the winding deformation will \nlead to partial discharges and gassing. Nevertheless, by the time partial discharge \nand subsequent gassing appear degradation of the transformer has already \nhappened. Winding deformation is one of the fi rst and basic precursors to show a \ndegraded condition in the transformer windings. The FRA tests are completed at the \nfactory and also in the fi eld. The measurements are cross compared to check if \nchanges have happened in the transformer. Special test equipment is needed to \ncomplete FRA or SFRA tests. Typically, the procedure requires the transformer to be\n\npage_label: 25\nfile_path: /kaggle/working/Documents/Transformer Testing Manual.pdf\n\ninterval is once every 5 min. This setting can be changed from once a minute, to \nonce per d ay. The sensors are set into the graphical user interface by the user \nduring installation process. Each monitored element is entere d into the datab ase \nwith an acquisition channel number, and descriptive text. The status and condition of \neach monitored element is shown on the monitor screen. \nFRA: National Electric Energy Testing, Research and A pplication Centre \n(NEETRAC) developed a technique to complete this test whil e the transformer is in \nservice. Online FRA data can give an up -to-date condition assessment of big, \nimportant transformers. Online FRA gives yet another tool to assess the physical \nstructure of the coils and their dielectric surroundings while t he transformer remains \nenergized. NEETRAC’ s online FRA technique uses typical system-switching \noperations, such as capacit or bank and reactor operations, along with lightning from \nlocal thunderstorms for the FRA test -signal s ource. NEETRAC’s patented method \ncan complete FRA signatures on transformer windings using a variety of input \nwaveforms with different time and amplitude shapes.\n---------------------\nGiven the context information and not prior knowledge, answer the query.]",- Core displacement \n- Partial winding collapse \n- Faulty core grounds \n- Shorted turns and open windings \n- Broken or loosened clamping structures \n- Winding deformation and displacement,"- Core displacement,\n- Partial winding collapse,\n- Faulty core grounds,\n- Shorted turns and open windings,\n- Broken or loosened 

In [29]:
import pandas as pd

# Combine the DataFrames
df_combined = pd.concat([df_1, df_2, df_3, df_4, df_5], ignore_index=True)

# Select the last 4 columns
last_4_columns = df_combined.iloc[:, -4:]

# Calculate the mean for each of the last 4 columns
averages = last_4_columns.mean()

# Print the averages with the column headers
for column, avg in averages.items():
    print(f"Average of {column}: {avg}")


Average of faithfulness: 0.9
Average of answer_relevancy: 0.8594405514846691
Average of context_precision: 0.9999999999
Average of context_recall: 1.0


# RAG-Evaluator

In [24]:
!pip install rag-evaluator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.0 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=60b9e4afd9b5b83d464aab98fc718bca5aa3e0ab320602019acdb854d74ca52b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [25]:
from rag_evaluator import RAGEvaluator

# Initialize the evaluator
evaluator = RAGEvaluator()
# Assuming evaluator is already defined and works with evaluate_all()

# Loop through the questions, answers, and ground_truths
for question, response, reference in zip(questions, answers, ground_truths):
    # Evaluate the response
    metrics = evaluator.evaluate_all(question, response, reference)
    
    # Print the results
    print(metrics)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

{'BLEU': 43.937095446369234, 'ROUGE-1': 0.7058823529411764, 'BERT P': 0.8589792251586914, 'BERT R': 0.9564086198806763, 'BERT F1': 0.9050794243812561, 'Perplexity': 112.91619873046875, 'Diversity': 0.9523809523809523, 'Racial Bias': 0.47735467553138733}
Using default tokenizer.
Using default tokenizer.
{'BLEU': 100.00000000000004, 'ROUGE-1': 1.0, 'BERT P': 1.0, 'BERT R': 1.0, 'BERT F1': 1.0, 'Perplexity': 119.43246459960938, 'Diversity': 0.95, 'Racial Bias': 0.4948898255825043}
Using default tokenizer.
Using default tokenizer.
{'BLEU': 11.239232901099346, 'ROUGE-1': 0.6467661691542289, 'BERT P': 0.7937517762184143, 'BERT R': 0.7974022030830383, 'BERT F1': 0.7955727577209473, 'Perplexity': 33.52040481567383, 'Diversity': 0.9213483146067416, 'Racial Bias': 0.5026652216911316}
Using default tokenizer.
Using default tokenizer.
{'BLEU': 53.03784722436516, 'ROUGE-1': 0.7027027027027027, 'BERT P': 0.8752954006195068, 'BERT R': 0.9561979174613953, 'BERT F1': 0.913959801197052, 'Perplexity': 73

# DeepEval

In [31]:
!pip install -q -q llama-index
!pip install -U -q deepeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.7/261.7 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.3/565.3 kB 14.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21

API- d4WwMdPM1ShsyKF/aFg5rd8Z5/RrtLJVCCZZaXrUu2I=

In [ ]:
!deepeval login --confident-api-key DeepEval Key


🎉🥳 Congratulations! You've successfully logged in! 🙌 
You're now using DeepEval with Confident AI. Follow our quickstart tutorial here: 
]8;id=722115;https://docs.confident-ai.com/confident-ai/confident-ai-introduction\https://docs.confident-ai.com/confident-ai/confident-ai-introduction]8;;\
E0000 00:00:1741605040.936807     319 init.cc:229] grpc_wait_for_shutdown_with_timeout() timed out.


In [44]:
groq_llm = ChatGroq(temperature=0, model_name="gemma2-9b-it", api_key=groq_api)

In [46]:
from deepeval import evaluate
from deepeval.metrics import (
    AnswerRelevancyMetric,
    ContextualRelevancyMetric,
    FaithfulnessMetric,
)
from deepeval.test_case import LLMTestCase
from langchain_groq import ChatGroq
from deepeval.models.base_model import DeepEvalBaseLLM

# Create custom DeepEval model using ChatGroq
class GroqModel(DeepEvalBaseLLM):
    def __init__(self, model):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "gemma2-9b-it"

# Create test cases from your data
test_cases = [
    LLMTestCase(
        input=question,
        actual_output=answer,
        expected_output=truth,
        retrieval_context=context
    ) for question, answer, truth, context in zip(questions[0:2], answers[0:2], ground_truths[0:2], contexts[0:2])
]

# Initialize all metrics
metrics = [
    AnswerRelevancyMetric(
        model=GroqModel(groq_llm),
        threshold=0.5,
        #evaluation_params=["input", "context"]
    ),
    ContextualRelevancyMetric(
        model=GroqModel(groq_llm),
        threshold=0.6,
        #evaluation_params=["input", "retrieval_context"]
    ),
    FaithfulnessMetric(
        model=GroqModel(groq_llm),
        threshold=0.7,
        #evaluation_params=["input", "context"]
    ),
]

# Run evaluation
if __name__ == "__main__":
    evaluate(
        test_cases,
        metrics=metrics,
        max_concurrent=2,
        #display_progress=True
    )

✨ You're running DeepEval's latest Answer Relevancy Metric! (using llama-3.3-70b-specdec, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using llama-3.3-70b-specdec, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using llama-3.3-70b-specdec, strict=False, 
async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 2 test case(s) in parallel: |          |  0% (0/2) [Time Taken: 00:00, ?test case/s]

...

Evaluating 2 test case(s) in parallel: |█████     | 50% (1/2) [Time Taken: 00:01,  1.76s/test case]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completion

Evaluating 2 test case(s) in parallel: |██████████|100% (2/2) [Time Taken: 00:02,  1.15s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: llama-3.3-70b-specdec, reason: The score is 1.00 because the output directly and accurately addresses the user's question about the purpose of an AC hi-pot test., error: None)
  - ✅ Contextual Relevancy (score: 0.9, threshold: 0.6, strict: False, evaluation model: llama-3.3-70b-specdec, reason: The score is 0.90 because while the retrieval context discusses the AC hi-pot test, it also includes information about the TTR test, which is irrelevant to the input.  For example,  it states  'This statement is about the TTR test and not the AC hi-pot test.' However, the context does provide relevant details about the AC hi-pot test, such as 'The AC hi -pot test is used to assess transformer windings condition.', error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: llama-3.3-70b-specdec, reason: The output is perfectly aligned with the context! Great j

✓ Tests finished 🎉! View results on 
]8;id=611177;https://app.confident-ai.com/project/cm82w692k1ixjqz0kncms1bkj/evaluation/test-runs/cm82z0nq61l8oqz0k6gwch4ug/compare-test-results\https://app.confident-ai.com/project/cm82w692k1ixjqz0kncms1bkj/evaluation/test-runs/cm82z0nq61l8oqz0k6gwch4ug/compa]8;;\
]8;id=611177;https://app.confident-ai.com/project/cm82w692k1ixjqz0kncms1bkj/evaluation/test-runs/cm82z0nq61l8oqz0k6gwch4ug/compare-test-results\re-test-results]8;;\.

In [49]:
from deepeval import evaluate
from deepeval.metrics import (
    AnswerRelevancyMetric,
    ContextualRelevancyMetric,
    FaithfulnessMetric,
)
from deepeval.test_case import LLMTestCase
from langchain_groq import ChatGroq
from deepeval.models.base_model import DeepEvalBaseLLM

# Create custom DeepEval model using ChatGroq
class GroqModel(DeepEvalBaseLLM):
    def __init__(self, model):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "gemma2-9b-it"

# Create test cases from your data
test_cases = [
    LLMTestCase(
        input=question,
        actual_output=answer,
        expected_output=truth,
        retrieval_context=context
    ) for question, answer, truth, context in zip(questions[2:4], answers[2:4], ground_truths[2:4], contexts[2:4])
]

# Initialize all metrics
metrics = [
    AnswerRelevancyMetric(
        model=GroqModel(groq_llm),
        threshold=0.5,
        #evaluation_params=["input", "context"]
    ),
    ContextualRelevancyMetric(
        model=GroqModel(groq_llm),
        threshold=0.6,
        #evaluation_params=["input", "retrieval_context"]
    ),
    FaithfulnessMetric(
        model=GroqModel(groq_llm),
        threshold=0.7,
        #evaluation_params=["input", "context"]
    ),
]

# Run evaluation
if __name__ == "__main__":
    evaluate(
        test_cases,
        metrics=metrics,
        max_concurrent=2,
        #display_progress=True
    )

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gemma2-9b-it, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gemma2-9b-it, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gemma2-9b-it, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 2 test case(s) in parallel: |          |  0% (0/2) [Time Taken: 00:00, ?test case/s]

...

Evaluating 2 test case(s) in parallel: |█████     | 50% (1/2) [Time Taken: 00:01,  1.41s/test case]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Evaluating 2 test case(s) in parallel: |██████████|100% (2/2) [Time Taken: 00:01,  1.28test case/s]



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: gemma2-9b-it, reason: The score is 1.00 because the output directly and accurately addresses the user's question about why AC voltage is preferred for transformer testing., error: None)
  - ✅ Contextual Relevancy (score: 1.0, threshold: 0.6, strict: False, evaluation model: gemma2-9b-it, reason: The retrieval context perfectly explains why AC voltage is preferred for transformer testing, stating that 'AC voltage is preferable to DC voltage for transformer testing because AC voltage simulates the internal stress that the transformers face during operating conditions.', error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: gemma2-9b-it, reason: The output is perfectly aligned with the context! Great job!, error: None)

For test case:

  - input: Why AC voltage is preferred for transformer testing?
  - actual output: AC voltage is preferable to DC

✓ Tests finished 🎉! View results on 
]8;id=426942;https://app.confident-ai.com/project/cm82w692k1ixjqz0kncms1bkj/evaluation/test-runs/cm82z3w021er44e3zjspfcxel/compare-test-results\https://app.confident-ai.com/project/cm82w692k1ixjqz0kncms1bkj/evaluation/test-runs/cm82z3w021er44e3zjspfcxel/compa]8;;\
]8;id=426942;https://app.confident-ai.com/project/cm82w692k1ixjqz0kncms1bkj/evaluation/test-runs/cm82z3w021er44e3zjspfcxel/compare-test-results\re-test-results]8;;\.

In [48]:
from deepeval import evaluate
from deepeval.metrics import (
    AnswerRelevancyMetric,
    ContextualRelevancyMetric,
    FaithfulnessMetric,
)
from deepeval.test_case import LLMTestCase
from langchain_groq import ChatGroq
from deepeval.models.base_model import DeepEvalBaseLLM

# Create custom DeepEval model using ChatGroq
class GroqModel(DeepEvalBaseLLM):
    def __init__(self, model):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "gemma2-9b-it"

# Create test cases from your data
test_cases = [
    LLMTestCase(
        input=question,
        actual_output=answer,
        expected_output=truth,
        retrieval_context=context
    ) for question, answer, truth, context in zip(questions[4:5], answers[4:5], ground_truths[4:5], contexts[4:5])
]

# Initialize all metrics
metrics = [
    AnswerRelevancyMetric(
        model=GroqModel(groq_llm),
        threshold=0.5,
        #evaluation_params=["input", "context"]
    ),
    ContextualRelevancyMetric(
        model=GroqModel(groq_llm),
        threshold=0.6,
        #evaluation_params=["input", "retrieval_context"]
    ),
    FaithfulnessMetric(
        model=GroqModel(groq_llm),
        threshold=0.7,
        #evaluation_params=["input", "context"]
    ),
]

# Run evaluation
if __name__ == "__main__":
    evaluate(
        test_cases,
        metrics=metrics,
        max_concurrent=2,
        #display_progress=True
    )

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gemma2-9b-it, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gemma2-9b-it, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gemma2-9b-it, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |          |  0% (0/1) [Time Taken: 00:00, ?test case/s]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completion

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:02,  2.96s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: gemma2-9b-it, reason: The score is 1.00 because the output is perfectly relevant to the input question about common internal mechanical issues in transformers with FRA., error: None)
  - ❌ Contextual Relevancy (score: 0.5862068965517241, threshold: 0.6, strict: False, evaluation model: gemma2-9b-it, reason: The score is 0.59 because while the retrieval context does contain relevant statements about common internal mechanical issues found in transformers with FRA,  like  "- Core displacement", "- Partial winding collapse", and "- Faulty core grounds", a significant portion of the context focuses on the FRA tool itself and its functionalities rather than the mechanical issues it identifies. , error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: gemma2-9b-it, reason: The output is perfectly aligned with the context! Great job!, error: None)

For 

✓ Tests finished 🎉! View results on 
]8;id=251441;https://app.confident-ai.com/project/cm82w692k1ixjqz0kncms1bkj/evaluation/test-runs/cm82z2w6r1lagqz0kdaxi4ae0/compare-test-results\https://app.confident-ai.com/project/cm82w692k1ixjqz0kncms1bkj/evaluation/test-runs/cm82z2w6r1lagqz0kdaxi4ae0/compa]8;;\
]8;id=251441;https://app.confident-ai.com/project/cm82w692k1ixjqz0kncms1bkj/evaluation/test-runs/cm82z2w6r1lagqz0kdaxi4ae0/compare-test-results\re-test-results]8;;\.